# About dataset
* 99Acres dataset contains the data of different cities data is divided on the basis of cities
* I have only taken 2 cities to train the model so that it can be done in less computation

In [29]:
#Importing data of city gurgaon
import pandas as pd
gurgaon = pd.read_csv('/content/gurgaon_10k.csv')
gurgaon.head(5)

<ipython-input-29-c47b2308c701>:3: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  gurgaon = pd.read_csv('/content/gurgaon_10k.csv')


,PROP_ID,PHOTO_URL,MEDIUM_PHOTO_URL,PREFERENCE,DESCRIPTION,PROPERTY_TYPE,CITY,LOCALITY,TRANSACT_TYPE,OWNTYPE,...,LOCALITY_WO_CITY,profile,xid,metadata,location,BUILTUP_SQFT,SUPER_SQFT,COMMON_FURNISHING_ATTRIBUTES,QUALITY_SCORE,FURNISHING_ATTRIBUTES
0,O71585872,https://mediacdn.99acres.com/media1/22409/13/4...,https://mediacdn.99acres.com/media1/22409/13/4...,S,4 bhk flat for resale in alpha corp gurgaon one,Residential Apartment,Gurgaon,Sector 84 Gurgaon,1.0,1,...,Sector 84,"{'CONTACT_NAME': 'gaurav khosla', 'CONTACT_COM...","{'PROJECTID': 5096, 'PROJ_NAME': 'Alpha Corp G...","{'PROP_PHOTO_COUNT': '4', 'PROP_VIDEO_COUNT': ...","{'CITY': '8', 'CITY_NAME': 'Gurgaon', 'BUILDIN...",NaN,NaN,NaN,NaN,NaN
1,T71585466,https://mediacdn.99acres.com/media1/22425/6/44...,https://mediacdn.99acres.com/media1/22425/6/44...,S,"Explore dlf ultima, an upscale ready-To-Move h...",Residential Apartment,Gurgaon,Sector 81 Gurgaon,1.0,1,...,Sector 81,"{'CONTACT_NAME': 'Kamal', 'CONTACT_COMPANY_NAM...","{'PROJECTID': 7182, 'PROJ_NAME': 'DLF The Ulti...","{'PROP_PHOTO_COUNT': '4', 'PROP_VIDEO_COUNT': ...","{'CITY': '8', 'CITY_NAME': 'Gurgaon', 'BUILDIN...",NaN,NaN,NaN,NaN,NaN
2,D71549524,https://mediacdn.99acres.com/media1/22386/13/4...,https://mediacdn.99acres.com/media1/22386/13/4...,R,A 3 bhk flat for rent in sector 66 gurgaon clo...,Residential Apartment,Gurgaon,Sector 66 Gurgaon,NaN,0,...,Sector 66,"{'CONTACT_NAME': 'Sandeep', 'CONTACT_COMPANY_N...","{'PROJECTID': 19, 'PROJ_NAME': 'Emaar MGF The ...","{'PROP_PHOTO_COUNT': '1', 'PROP_VIDEO_COUNT': ...","{'CITY': '8', 'CITY_NAME': 'Gurgaon', 'BUILDIN...",1920.00,NaN,NaN,NaN,NaN
3,I71493958,https://mediacdn.99acres.com/media1/22353/14/4...,https://mediacdn.99acres.com/media1/22353/14/4...,S,Looking for a 3 bhk property for purchase in g...,Residential Apartment,Gurgaon,Sector 112 Gurgaon,1.0,1,...,Sector 112,"{'CONTACT_NAME': 'Kohli', 'CONTACT_COMPANY_NAM...","{'PROJECTID': 4342, 'PROJ_NAME': 'Experion Win...","{'PROP_PHOTO_COUNT': '1', 'PROP_VIDEO_COUNT': ...","{'CITY': '8', 'CITY_NAME': 'Gurgaon', 'BUILDIN...",2225.56,NaN,NaN,NaN,NaN
4,H71479412,https://mediacdn.99acres.com/media1/22352/4/44...,https://mediacdn.99acres.com/media1/22352/4/44...,R,This 4 bhk flat in sector 104 gurgaon is avail...,Residential Apartment,Gurgaon,Sector 104 Gurgaon,NaN,0,...,Sector 104,"{'CONTACT_NAME': 'Kohli', 'CONTACT_COMPANY_NAM...","{'PROJECTID': 25860, 'PROJ_NAME': 'ATS Triumph...","{'PROP_PHOTO_COUNT': '14', 'PROP_VIDEO_COUNT':...","{'CITY': '8', 'CITY_NAME': 'Gurgaon', 'BUILDIN...",NaN,NaN,NaN,NaN,NaN


In [30]:
# Dropping useless column
gurgaon.drop(columns = ['PHOTO_URL', 'MEDIUM_PHOTO_URL', 'EXPIRY_DATE', 'PD_URL', 'PROPERTY_IMAGES', 'THUMBNAIL_IMAGES', 'CONTACT_NAME', 'CONTACT_COMPANY_NAME', 'DEALER_PHOTO_URL', 'profile', 'xid', 'metadata', 'location'], inplace = True)

In [31]:
# Dropping useless column
gurgaon.drop(columns = ['COMMON_FURNISHING_ATTRIBUTES','QUALITY_SCORE','FURNISHING_ATTRIBUTES','BUILDING_ID', 'PROP_ID','PROP_DETAILS_URL','FSL_Data','TRANSACT_TYPE'], inplace = True)

In [32]:
# Cleaning Locality column
gurgaon['LOCALITY'] = gurgaon['LOCALITY'].str.split('Gurgaon').str.get(0).str.strip()

In [33]:
# function used to find the BHK of house
def get_bhk(text):
  pattern = r'(\d+)(?=\s*bhk)'
  x = list(set(re.findall(pattern, text)))
  if len(x) == 0:
    return None
  else:
    return x[0]

In [34]:
import re
text = gurgaon['DESCRIPTION'].apply(lambda x: get_bhk(x))
gurgaon['BHK'] = text

In [35]:
gurgaon['BHK'] = gurgaon['BHK'].astype(float)

In [36]:
# Extracting longitude and latitude From Map_details
import ast
Lat = []
Long = []
for d in gurgaon['MAP_DETAILS']:
    lat = ast.literal_eval(d)['LATITUDE']
    long = ast.literal_eval(d)['LONGITUDE']
    Lat.append(lat)
    Long.append(long)

In [37]:
gurgaon['LATITUDE'] = pd.Series(Lat).astype(float)
gurgaon['LONGITUDE'] = pd.Series(Long).astype(float)

In [38]:
# Cleaning Area column
gurgaon['AREA'] = gurgaon['AREA'].str.split(' ', expand=True).get(0)

In [39]:
# Dropping useless column
gurgaon.drop(columns = ['CITY_ID','PREFERENCE','DESCRIPTION','REGISTER_DATE','LISTING','BROKERAGE','FORMATTED','TOP_USPS','GROUP_NAME'], inplace = True)

In [40]:
indx = gurgaon[gurgaon['PRICE'] == 'Price on Request']

In [41]:
gurgaon.drop(indx.index, inplace=True)

In [42]:
# function to refine Price data
def price(item):
  if 'L' in item:
    return float(item.replace('L', '').replace('.','').strip())*100000
  elif 'Cr' in item:
    return float(item.replace('Cr', '').strip())*10000000
  elif ',' in item:
    return float(item.replace(',','').strip())
  else:
    return float(item)

In [43]:
# function to refine Price data
def p(item):
  if '-' in item:
    return item.split('-')[1]
  else:
    return item

In [44]:
gurgaon['PRICE'] = gurgaon['PRICE'].apply(p).apply(price)

In [45]:
# Dropping useless column
gurgaon.drop(columns = ['MIN_PRICE','MAX_PRICE','MIN_AREA_SQFT','MAX_AREA_SQFT','PROP_HEADING','CLASS_HEADING','CLASS_LABEL','SECONDARY_TAGS','TOTAL_LANDMARK_COUNT','FORMATTED_LANDMARK_DETAILS','SOCIETY_NAME','BUILDING_NAME','LOCALITY_WO_CITY','BUILTUP_SQFT','SUPER_SQFT'], inplace = True)

In [46]:
# Dropping useless column
gurgaon.drop(columns = ['MAP_DETAILS'], inplace = True)

In [47]:
# Dropping useless column
gurgaon.drop(columns=['PRICE_PER_UNIT_AREA'],inplace = True)

In [48]:
# calculating total sccore of amenities
def calculate_luxury_score(amenities):
    # Define amenity scores (adjust as needed)
    amenity_scores = {
        'Waste Disposal' : 1,
        'Rain Water Harvesting' : 3,
        'Bank Attached Property' : 2,
        'Power Back-up' : 4,
        'Feng Shui / Vaastu Compliant' : 5,
        'Private Garden / Terrace' : 8,
        'Centrally Air Conditioned' : 9,
        'Security / Fire Alarm' : 7,
        'Intercom Facility' : 6,
        'Water Storage' : 2,
        'Piped-gas' : 3,
        'Water purifier' : 4,
        'Near Bank' : 1,
        'Swimming Pool' : 9,
        'Club house / Community Center' : 8,
        'Park' : 5,
        'Security Personnel' : 7,
        'Fitness Centre / GYM' : 8,
        'Visitor Parking' : 4,
        'Lift(s)' : 9,
        'Maintenance Staff' : 6,
        'Shopping Centre' : 7,
        'WheelChair Accessibility' : 5,
        'DG Availability' : 8,
        'CCTV Surveillance' : 6,
        'Grade A Building' : 9,
        'Grocery Shop' : 3,
        'ATM' : 1,
        'Cafeteria / Food Court' : 7,
        'Bar / Lounge' : 9,
        'Conference room' : 8,
        'Service / Goods Lift' : 6,
        'Access to High Speed Internet' : 9
        # Add other amenities and their scores here
    }

    # Calculate total luxury score for the property
    total_score = sum(amenity_scores.get(amenity, 1) for amenity in amenities)

    return total_score

def categorize_property(total_score):
    if total_score >= 15:
        return "Luxury"
    elif 8 <= total_score <= 14:
        return "Semi-luxury"
    else:
        return "Budget"


luxury_score = gurgaon['PROPERTY_TYPE'].apply(calculate_luxury_score)
gurgaon['LUXURY'] = luxury_score.apply(categorize_property)
print(f"Total Luxury Score: {luxury_score}")
print(f"Category: {gurgaon['LUXURY']}")

Total Luxury Score: 0        21
1        21
2        21
3        21
4        21
         ..
10699    21
10700    21
10701    21
10702    21
10703    25
Name: PROPERTY_TYPE, Length: 10682, dtype: int64
Category: 0        Luxury
1        Luxury
2        Luxury
3        Luxury
4        Luxury
          ...  
10699    Luxury
10700    Luxury
10701    Luxury
10702    Luxury
10703    Luxury
Name: LUXURY, Length: 10682, dtype: object


In [49]:
# Dropping useless column
gurgaon.drop(columns = ['AMENITIES'], inplace = True)

In [50]:
# Dropping useless column
gurgaon.drop(columns=['PROP_NAME'],inplace = True)

In [51]:
# Dropping useless column
gurgaon.drop(columns=['BATHROOM_NUM'], inplace=True)

In [52]:
# Dropping duplicates
gurgaon.drop_duplicates(inplace = True)

In [53]:
gurgaon[(gurgaon['CARPET_SQFT'] == None) & (gurgaon['SUPERBUILTUP_SQFT'] == None)]

,PROPERTY_TYPE,CITY,LOCALITY,OWNTYPE,BEDROOM_NUM,BALCONY_NUM,FURNISH,FACING,AGE,FLOOR_NUM,...,FEATURES,PRICE_SQFT,CARPET_SQFT,SUPERBUILTUP_SQFT,AREA,PRICE,BHK,LATITUDE,LONGITUDE,LUXURY


In [54]:
gurgaon.to_csv('gurgaon_cleaned.csv',index = True)

In [55]:
gurgaon.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10671 entries, 0 to 10703
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PROPERTY_TYPE      10671 non-null  object 
 1   CITY               10671 non-null  object 
 2   LOCALITY           10671 non-null  object 
 3   OWNTYPE            10671 non-null  int64  
 4   BEDROOM_NUM        10479 non-null  float64
 5   BALCONY_NUM        10472 non-null  float64
 6   FURNISH            10671 non-null  int64  
 7   FACING             10671 non-null  int64  
 8   AGE                10671 non-null  int64  
 9   FLOOR_NUM          9777 non-null   object 
 10  TOTAL_FLOOR        10664 non-null  float64
 11  FEATURES           10671 non-null  object 
 12  PRICE_SQFT         10671 non-null  float64
 13  CARPET_SQFT        6526 non-null   float64
 14  SUPERBUILTUP_SQFT  6689 non-null   float64
 15  AREA               10671 non-null  object 
 16  PRICE              10671 no

In [56]:
# am = pd.read_excel('/content/amenities.xlsx')
# # am.reset_index(drop=True, inplace=True)
# am.set_index('id')

In [57]:
import pandas as pd

In [58]:
pd.set_option('display.max_columns', None)

In [59]:
pd.set_option('display.max_rows', None)

In [60]:
# importing hyderbad dataset
hyderbad = pd.read_csv('/content/hyderabad.csv')
hyderbad.head(1)

,SPID,PROP_ID,PREFERENCE,DESCRIPTION,PROPERTY_TYPE,CITY,TRANSACT_TYPE,OWNTYPE,BEDROOM_NUM,PRICE_PER_UNIT_AREA,FURNISH,FACING,AGE,TOTAL_FLOOR,FEATURES,REGISTER_DATE,POSTING_DATE,UPDATE_DATE,CLASS,RES_COM,PROP_NAME,PROPERTY_NUMBER,MIN_PRICE,MAX_PRICE,PRICE_SQFT,PROJ_ID,BUILDING_ID,VERIFIED,MAP_DETAILS,MIN_AREA_SQFT,MAX_AREA_SQFT,AMENITIES,ALT_TAG,PRODUCT_TYPE,TOP_USPS,EXPIRY_DATE,PROPERTY_TYPE__U,AREA,SECONDARY_AREA,PRICE,PROP_HEADING,VALUE_LABEL,CLASS_HEADING,CLASS_LABEL,REGISTER_DATE__U,REGISTERED_DAYS,PRIMARY_TAGS,SECONDARY_TAGS,TOTAL_LANDMARK_COUNT,FORMATTED_LANDMARK_DETAILS,SOCIETY_NAME,BUILDING_NAME,location,BALCONY_NUM,FLOOR_NUM
0,70135726,Z70135726,S,"Hmda, rera approved residential apartments at ...",Residential Apartment,Secunderabad,1.0,1,2.0,5700.0,2,4,1,5.0,"23,24,5,17,6,20,21","08th Jul, 2023",1.688834e+12,1689100520000,A,R,Evolve,1,6925500,6925500,5700.0,0,0,N,"{'LATITUDE': '17.575957', 'LONGITUDE': '78.471...",112.877145,112.877145,"5,17,20,21,23,24,6","₹69.25 Lac, 2 bhk Residential Apartment in Gun...",PREMIUM_LISTING,NaN,2023-09-06 00:00:00,1,1215 sq.ft.,112.88 sq.m.,69.25 L,2 BHK Flat in Gundlapochampalli,Freehold,Dealer: Rainbow Developers,Dealer,2023-07-08 22:01:14,1 month ago,['FEATURED'],"['READY TO MOVE', 'RESALE']",9.0,[{'text': '1 Library'}],Evolve,Evolve,"{'CITY': '268', 'CITY_NAME': 'Secunderabad', '...",1.0,2


In [61]:
#dropping all empty rows in the column AMENITES
hyderbad = hyderbad[hyderbad['AMENITIES'].notna()]

In [62]:
import re
def bhk(text):
  pattern = r'(\d+)(?=\s*BHK)'
  x = re.findall(pattern, text)
  if len(x) == 0:
    return None
  else:
    return x

In [63]:
hyderbad['BHK'] = hyderbad['PROP_HEADING'].apply(bhk).str.get(0)

In [64]:
import ast
text = []
for i in hyderbad['location']:
  text = (ast.literal_eval(i)['LOCALITY_NAME'])
hyderbad['LOCALITY'] = text

In [65]:
import ast
Lat = []
Long = []
for d in hyderbad['MAP_DETAILS']:
    lat = ast.literal_eval(d)['LATITUDE']
    long = ast.literal_eval(d)['LONGITUDE']
    Lat.append(lat)
    Long.append(long)

In [66]:
hyderbad['LATITUDE'] = pd.Series(Lat).astype(float)
hyderbad['LONGITUDE'] = pd.Series(Long).astype(float)

In [67]:
hyderbad['AMENITIES'] = hyderbad['AMENITIES'].str.split(',').tolist()

In [68]:
def calculate_luxury_score(amenities):
    # Define amenity scores (adjust as needed)
    amenity_scores = {
        '1' : 1,
        '2' : 3,
        '3' : 2,
        '4' : 4,
        '5' : 5,
        '6' : 8,
        '7' : 9,
        '8' : 7,
        '9' : 6,
        '10' : 2,
        '11' : 3,
        '12' : 4,
        '13' : 1,
        '14' : 9,
        '15' : 8,
        '16' : 5,
        '17' : 7,
        '18' : 8,
        '19' : 4,
        '20' : 9,
        '21' : 6,
        '22' : 7,
        '23' : 5,
        '24' : 8,
        '25' : 6,
        '26' : 9,
        '27' : 3,
        '28' : 1,
        '29' : 7,
        '30' : 9,
        '31' : 8,
        '32' : 6,
        '33' : 9
        # Add other amenities and their scores here
    }

    # Calculate total luxury score for the property
    if amenities == None:
      return 1

    total_score = sum(amenity_scores.get(amenity, 1) for amenity in amenities)

    # for i in range(len(amenities)):
    #   total_score += amenity_scores[amenities[i]]

    return total_score

def categorize_property(total_score):
    if total_score >= 15:
        return "Luxury"
    elif 8 <= total_score <= 14:
        return "Semi-luxury"
    else:
        return "Budget"


luxury_score = hyderbad['AMENITIES'].apply(calculate_luxury_score)
hyderbad['LUXURY'] = luxury_score.apply(categorize_property)
# print(f"Total Luxury Score: {luxury_score}")
# print(f"Category: {['LUXURY']}")

In [69]:
hyderbad.rename(columns = {'SECONDARY_AREA' : 'SUPERBUILTUP_SQFT'}, inplace=True)

In [70]:
hyderbad['AREA'] = hyderbad['AREA'].str.split(' ', expand=True).get(0).astype(float)

In [71]:
hyderbad['SUPERBUILTUP_SQFT'] = hyderbad['SUPERBUILTUP_SQFT'].str.split(' ', expand=True).get(0).astype(float)*10.7639104

In [72]:
hyderbad['CARPET_SQFT'] = hyderbad['AREA'] - abs(hyderbad['AREA'] - hyderbad['SUPERBUILTUP_SQFT'])

In [73]:
hyderbad.drop(columns=['AMENITIES','PRICE_PER_UNIT_AREA','location','MAP_DETAILS','DESCRIPTION','PREFERENCE','SPID','PROP_ID','TRANSACT_TYPE','REGISTER_DATE','POSTING_DATE','UPDATE_DATE','CLASS','RES_COM','PROP_NAME','PROPERTY_NUMBER','MIN_PRICE','MAX_PRICE','PROJ_ID','BUILDING_ID','VERIFIED','MIN_AREA_SQFT','MAX_AREA_SQFT','PRODUCT_TYPE','TOP_USPS','EXPIRY_DATE','PROPERTY_TYPE__U','ALT_TAG','PROP_HEADING','VALUE_LABEL','CLASS_HEADING','CLASS_LABEL','REGISTER_DATE__U','REGISTERED_DAYS','PRIMARY_TAGS','SECONDARY_TAGS','TOTAL_LANDMARK_COUNT','FORMATTED_LANDMARK_DETAILS','SOCIETY_NAME','BUILDING_NAME'], inplace=True)

In [74]:
hyderbad = hyderbad[hyderbad['PRICE'].str.contains('Bed') == False]

In [75]:
hyderbad = hyderbad[hyderbad['PRICE'] != 'Price on Request']

In [76]:
def bed(item):
  return float(item['PRICE'].str.split('/').get(0))*item['BEDROOM_NUM']

In [77]:
hyderbad['PRICE'] = hyderbad['PRICE'].apply(price)

In [78]:
hyderbad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8058 entries, 0 to 9486
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PROPERTY_TYPE      8058 non-null   object 
 1   CITY               8058 non-null   object 
 2   OWNTYPE            8058 non-null   int64  
 3   BEDROOM_NUM        5812 non-null   float64
 4   FURNISH            8058 non-null   int64  
 5   FACING             8058 non-null   int64  
 6   AGE                8058 non-null   int64  
 7   TOTAL_FLOOR        7655 non-null   float64
 8   FEATURES           8058 non-null   object 
 9   PRICE_SQFT         8058 non-null   float64
 10  AREA               8058 non-null   float64
 11  SUPERBUILTUP_SQFT  8058 non-null   float64
 12  PRICE              8058 non-null   float64
 13  BALCONY_NUM        5812 non-null   float64
 14  FLOOR_NUM          4395 non-null   object 
 15  BHK                4372 non-null   object 
 16  LOCALITY           8058 non-n

In [79]:
hyderbad.to_csv('hyderbad_cleaned.csv',index = True)

In [80]:
data = pd.concat([hyderbad, gurgaon])

In [81]:
data.head(1)

,PROPERTY_TYPE,CITY,OWNTYPE,BEDROOM_NUM,FURNISH,FACING,AGE,TOTAL_FLOOR,FEATURES,PRICE_SQFT,AREA,SUPERBUILTUP_SQFT,PRICE,BALCONY_NUM,FLOOR_NUM,BHK,LOCALITY,LATITUDE,LONGITUDE,LUXURY,CARPET_SQFT
0,Residential Apartment,Secunderabad,1,2.0,2,4,1,5.0,"23,24,5,17,6,20,21",5700.0,1215.0,1215.030206,692500000.0,1.0,2,2,Manikonda,17.575957,78.471167,Luxury,1214.969794


In [96]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Create DataFrame
df = pd.DataFrame(data['FEATURES'])

# Split the string into lists of numbers
df['FEATURES'] = df['FEATURES'].apply(lambda x: x.split(','))

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Transform the data
encoded_df = pd.DataFrame(mlb.fit_transform(df['FEATURES']), columns=mlb.classes_)

encoded_df = encoded_df[['1','2','3','4','6','9','12','21','25','29','39']]

encoded_df.rename(columns = {
    '1' : 'SWIMMING_POOL',
    '2' : 'POWER_BACKUP',
    '3' : 'CLUB_HOUSE',
    '4' : 'PARKING',
    '12' : 'GYM',
    '6' : 'PARKING',
    '9' : 'ATM',
    '21' : 'LIFT',
    '25' : 'WASTE_DISPOSAL',
    '29' : 'GAS_PIPELINE',
    # '34' : 'WHEELCHAIR_ACCESSIBILITY',
    # '35' : 'DG_AVAILABILITY',
    '39' : 'NEAR_BANK'
}, inplace=True)

data.reset_index(drop=True, inplace=True)
# Concatenate the encoded DataFrame with the original DataFrame
data = pd.concat([data, encoded_df], axis=1, ignore_index=False)
# Display the encoded DataFrame
data.head(1)


,PROPERTY_TYPE,CITY,OWNTYPE,BEDROOM_NUM,FURNISH,FACING,AGE,TOTAL_FLOOR,FEATURES,PRICE_SQFT,AREA,SUPERBUILTUP_SQFT,PRICE,BALCONY_NUM,FLOOR_NUM,BHK,LOCALITY,LATITUDE,LONGITUDE,LUXURY,CARPET_SQFT,SWIMMING_POOL,POWER_BACKUP,CLUB_HOUSE,PARKING,PARKING,9,GYM,LIFT,WASTE_DISPOSAL,GAS_PIPELINE,NEAR_BANK,SWIMMING_POOL,POWER_BACKUP,CLUB_HOUSE,PARKING,PARKING,ATM,GYM,LIFT,WASTE_DISPOSAL,GAS_PIPELINE,NEAR_BANK
0,Residential Apartment,Secunderabad,1,2.0,2,4,1,5.0,"23,24,5,17,6,20,21",5700.0,1215.0,1215.030206,692500000.0,1.0,2,2,Manikonda,17.575957,78.471167,Luxury,1214.969794,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0


In [97]:
data.to_csv('99Acres_gurgaon_hyderabad.csv')